In [9]:
import numpy as np
import random
import time
import multiprocessing
import multiprocessing.pool
random.seed(time.time())

In [10]:
import os
dir1="base_matrices"
#sys.path.append("base_matrices")
np.loadtxt(os.path.join('base_matrices', 'NR_1_0_2.txt'))

array([[ 0.,  1.,  0., ..., -1., -1., -1.],
       [ 0., -1.,  1., ..., -1., -1., -1.],
       [ 0.,  1.,  1., ..., -1., -1., -1.],
       ...,
       [-1.,  1., -1., ...,  0., -1., -1.],
       [ 1., -1., -1., ..., -1.,  0., -1.],
       [-1.,  1., -1., ..., -1., -1.,  0.]])

In [11]:
def generate_information(K):
  #generate information
  information=np.random.randint(0,2,K)
  return information

In [23]:
def generate_filename(K,R):

    #decide BG_num
    if K<=3824 and R<=0.67:
        BG_num=2
    elif K<=292:
        BG_num=2
    elif R<=0.25:
        BG_num=2
    else:
        BG_num=1

    #decide Kb
    if BG_num==1:
        Kb=22
    else:
        if K>640:
            Kb=10
        elif 560<K<=640:
            Kb=9
        elif 192<K<=560:
            Kb=8
        elif K<=192:
            Kb=6

    #decide Zc

    a=np.arange(2,16)
    j=np.arange(0,8)


    a,j=np.meshgrid(a,j)
    a=a.flatten()
    j=j.flatten()

    Zc_array=a*(2**j)
    MAX_Zc=384
    Zc_array=Zc_array[MAX_Zc>=Zc_array]
    Zc=np.min(Zc_array[Zc_array>=K/Kb])

    #decide iLS
    i=list()
    i0=np.array([2,4,8,16,32,64,128,256])
    i1=np.array([3,6,12,24,48,96,192,384])
    i2=np.array([5,10,20,40,80,160,320])
    i3=np.array([7,14,28,56,112,224])
    i4=np.array([9,18,36,72,144,288])
    i5=np.array([11,22,44,88,176,352])
    i6=np.array([13,26,52,104,208])
    i7=np.array([15,30,60,120,240])
    i_list=[i0,i1,i2,i3,i4,i5,i6,i7]

    for count,i in enumerate(i_list):
        if np.any(i==Zc):
            iLS=count
    
    filename='NR_'+str(BG_num)+'_'+str(iLS)+'_'+str(Zc)+'.txt'

    return Zc,filename,BG_num,Kb

In [145]:
def permute(a,Zc): #n*n単位行列をaだけシフトさせる
    if a==-1:
        tmp=np.zeros((Zc,Zc),dtype=int)
    else:
        tmp=np.identity(Zc,dtype=int)
        tmp=np.roll(tmp,a,axis=1)
    return tmp

def generate_PC_MATRIX(BG_num,N,K,Kb,Zc,filename):

    base_matrix=np.loadtxt(os.path.join('base_matrices', filename),dtype='int')
    
    if BG_num==1:
        tmp=22
    elif BG_num==2:
        tmp=10
    
    Mb=np.arange((N-K)//Zc+1)
    Nb=np.zeros(Kb+2+len(Mb),dtype='int')
    Nb[:Kb+2]=np.arange(Kb+2)
    Nb[Kb+2:]=np.arange(tmp,tmp+len(Mb))
    #print(Nb)
    #print(Mb)

    PC_MATRIX=np.empty((0,Zc*len(Nb)),dtype=int)
    for i in Mb:
        matrix_row=np.empty((Zc,0),dtype=int)

        for j in Nb:
            tmp=permute(base_matrix[i,j],Zc)
            matrix_row=np.concatenate([matrix_row,tmp],axis=1)

        PC_MATRIX=np.concatenate([PC_MATRIX,matrix_row],axis=0)
        
    return PC_MATRIX

In [210]:
N=4096
K=2048
Zc,filename,BG_num,Kb=generate_filename(K,K/N)
print(filename)
PC_MATRIX=generate_PC_MATRIX(BG_num,N,K,Kb,Zc,filename)
np.savetxt("PC_MATRIX",PC_MATRIX,fmt='%i')
if PC_MATRIX[-1,-1]!=1:
  print("error!")


NR_2_6_208.txt


In [211]:


N=PC_MATRIX.shape[1]
K=PC_MATRIX.shape[1]-PC_MATRIX.shape[0]
print(N-2*Zc,K)

codeword=np.zeros(N,dtype=int)

#0:2*Zc:shortened code
codeword[2*Zc:K]=np.random.randint(0,2,(K-2*Zc))

#double diagonal structure
matrix=np.zeros((Zc,N))
for i in range(4):
    matrix=(matrix+PC_MATRIX[i*Zc:(i+1)*Zc])%2
#print(matrix.shape)

#first base-matrix-row parity check 
for i in range(K,K+Zc):
    count=0
    while count<Zc:        
        if matrix[count,i]==1:
            codeword[i]=codeword@np.transpose(matrix[count,:])%2
            #print(i,count)
            break
        count+=1
#K-K+Zcまでのparitybitを生成

#K+Zc-K+3Zcまでのparitybitを生成
for i in range(K+Zc,K+3*Zc):
    j=i-K-Zc
    codeword[i]=codeword@np.transpose(PC_MATRIX[j,:])%2
    #print(codeword[i])
    #print(i)
    #print(codeword[K:])
    #from IPython.core.debugger import Pdb; Pdb().set_trace()

#nomal structure
for i in range(K+3*Zc,N):
    j=i-K
    codeword[i]=codeword@np.transpose(PC_MATRIX[j,:])%2

if np.any(PC_MATRIX@codeword%2!=0):
    tmp=PC_MATRIX@codeword%2
    print(tmp[:3*Zc])
    print("cword err")

4160 2496


In [342]:
#nomal structure
for i in range(K+3*Zc,N):
    j=i-K
    codeword[i]=codeword@np.transpose(PC_MATRIX[j,:])%2
    #print(i)
    #print(codeword[K:])
    #from IPython.core.debugger import Pdb; Pdb().set_trace()

if np.any(PC_MATRIX@codeword%2!=0):
    tmp=PC_MATRIX@codeword%2
    print(tmp[:3*Zc])
    print("cword err")

    #return codeword

In [355]:
information=np.random.randint(0,2,PC_MATRIX.shape[1]-PC_MATRIX.shape[0])
print(len(information))
np.savetxt("PC_MATRIX",PC_MATRIX,fmt='%i')
codeword=encode(PC_MATRIX,information,Zc)

640
1216 640
0
matrix 1.0
1.0
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 0 1 0 0 1
 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0
 1 0 0 1 0 0 0]
cword err
